## Object Detection with YOLOv5

### About
----------------
This notebook provides a guided tour of deploying a YOLOv5 machine learning model pre-trained with MS COCO dataset using PyTorch to a Panorama appliance. More information about the model including the original model itself can be found in [this repository](https://github.com/ultralytics/yolov5). More specifically, release `v3.0` of the repository was used to build this example.

This example includes a model ready to be deployed to a Panorama device (TorchScript export of `yolov5s.pt` model). You can also train your own model using the resources from the aforementioned repository, and if you do that, make sure to export the resulting model to TorchScript (export script is also available in that repository) and pack it into a tarball (`.tar.gz`) before deploying it to the Panorama device.

This is an example of inference done on an image captured from a test IP camera

![alt Test image inference results](test-result.png "Test image inference results")

### Imports & config
----------------

***This notebook was tested with Torch v1.6.0, upgrade it in case you are running an older version***

Convert the next cell to Code and run it to install the tested version of PyTorch

In [ ]:
import time
import os
import random as rnd
import json

from matplotlib import pyplot as plt
import numpy as np
import cv2
import torch
import boto3

import utils
print(f'Using torch {torch.__version__}')

***Create your own AWS S3 Bucket making sure its name contains `aws-panorama`***

In [ ]:
# Set this variable/constant value to be the full name of your bucket, for example "aws-panorama-example-xyz"
BUCKET = 'aws-panorama-<you-bucket-name-suffix>'  # Bucket name must contain "aws-panorama"

# TEMP
BUCKET = 'aws-panorama-demo'

MODELS_S3_PREFIX = 'models'
models_s3_url = f's3://{BUCKET}/{MODELS_S3_PREFIX}/'
MODEL_SOURCE_URL = 'http://d3m09i4q1na4l7.cloudfront.net/yolov5s-640.tar.gz'

MODEL = 'yolov5s'
model_file = f'{MODEL}.pth'
model_archive = f'{MODEL}-640.tar.gz'

LAMBDA = 'yolov5s'

LAMBDA_EXECUTION_ROLE_NAME = 'PanoramaYoloLambdaExecutionRole'
lambda_file = f'{LAMBDA}_lambda.py'
lambda_archive = lambda_file.replace('.py', '.zip')

TEST_IMAGE = 'test.png'
INPUT_SIZE = 640 
THRESHOLD = 0.5
CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
           'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 
           'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 
           'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 
           'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
           'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
           'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 
           'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 
           'scissors', 'teddy bear', 'hair drier', 'toothbrush']

if torch.cuda.is_available():
    device_type = 'GPU'
    print(f'Using GPU: {torch.cuda.get_device_properties(0)}')
else:
    device_type = 'CPU'
    print(f'Using {device_type}')

device = torch.device('cuda:0' if device_type == 'GPU' else 'cpu')

In [ ]:
iam_client = boto3.client("iam")
lambda_client = boto3.client("lambda")

### The model
----------------

##### Test the modal locally
- Download the provided model archive
- Extract the model from the archive
- Load the model
- Load the test image and prepare it
- Put the test image through the model
- Show results

In [ ]:
!wget $MODEL_SOURCE_URL
!tar -xvf $model_archive

In [ ]:
model = torch.jit.load(model_file, map_location=device)

In [ ]:
test_image = cv2.cvtColor(cv2.imread(TEST_IMAGE), cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6, 6))
plt.imshow(test_image)

Note: output of the YOLOv5 model requires further processing (Non Max Suppression) which can be done on GPU using PyTorch but on a Panorama appliance it needs to be executed on CPU (also applies to execution of model's Detector layer logic), adding significant overhead to the overall inference process.

In [ ]:
processor = utils.Processor(CLASSES, INPUT_SIZE, keep_ratio=True)

tm = time.time()
img = torch.from_numpy(processor.preprocess(test_image)).to(device)
print(f'Pre-process: {int((time.time() - tm) * 1000)} msec')

# Do a warm-up run before timing inference
model(img)

tm = time.time()
results = model(img)
print(f'Inference: {int((time.time() - tm) * 1000)} msec')
test_results = [r.cpu().numpy() for r in results]

tm = time.time()
_, result_image = processor.post_process(test_results, test_image.shape, THRESHOLD, test_image.copy())
print(f'Post-process: {int((time.time() - tm) * 1000)} msec')

plt.figure(figsize=(6, 6))
plt.imshow(result_image)

##### Prepare the model
All we need to do is to upload the model archive to S3

In [ ]:
jit_model_s3_url = os.path.join(models_s3_url, model_archive)
!aws s3 cp $model_archive $jit_model_s3_url
!aws s3 ls $jit_model_s3_url --human-readable

### The Application 
---------------------

This is the script that will be deployed and executed on the Panorama Appliance as a lambda function. It can found in the same folder as this notebook along with another file `utils.py`, containing some helper scripts.

In [ ]:
!pygmentize $lambda_file

#### Create and deploy lambda

- If the execution of the code in this cell fails then make sure you have the rights to creates roles in AWS IAM
- You only need to execute the next cell once. All the subsequent deployments will use the same role 

In [ ]:
lambda_execution_role_policy = {
    "Version": "2012-10-17",
    "Statement":[
        {
            "Effect": "Allow",
            "Principal": {"Service": ["lambda.amazonaws.com", "events.amazonaws.com"]},
            "Action": "sts:AssumeRole",
        }
    ]
}
iam_client.create_role(
    RoleName=LAMBDA_EXECUTION_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(lambda_execution_role_policy),
)

##### Create a new function

*Use the cell after the next if you want to re-deploy lambda after the initial deployment*

You can inspect the created AWS Lambda Function following the link shown after running the next cell

In [ ]:
!zip -o $lambda_archive $lambda_file utils.py

In [ ]:
lambda_client.delete_function(
    FunctionName=LAMBDA)

In [ ]:
with open(lambda_archive, "rb") as f:
    zipped_code = f.read()
    
lambda_execution_role = iam_client.get_role(RoleName=LAMBDA_EXECUTION_ROLE_NAME)

lambda_response = lambda_client.create_function(
    FunctionName=LAMBDA,
    Runtime="python3.7",
    Role=lambda_execution_role["Role"]["Arn"],
    Handler=lambda_file.replace('.py', '.main()'),
    Code=dict(ZipFile=zipped_code),
    Timeout=120,  
    MemorySize=2048,
    Publish=True)

template = "https://console.aws.amazon.com/lambda/home?region=us-east-1#/functions/{}/versions/{}?tab=configuration"
lambda_url = template.format(lambda_response["FunctionName"], lambda_response["Version"])
print(lambda_url)

##### [OPTIONAL] Subsequent deployments
Convert the next cell to Code and run the following cell if you want to re-deploy the lambda function again, e.g. if you make changes to application code and want to deploy those changes to the Panorama appliance

### Deploy the Application to Panorama appliance 
---------------------

At the time of writing this the only way to deploy the Application to the Panorama device is through the [AWS Panorama Console](https://console.aws.amazon.com/panorama). Instructions for script-based deployment will be added here after alternative means of deployment are available, e.g. via AWS CLI or Python SDK.

Few things to remember when deploying the Application to your Panorama appliance in AWS Panorama Console:

- when creating a new model (as part of a new Application creation process) in AWS Panorama Console:
    - use the model archive uploaded to S3 earlier in this notebook to create a new **External model** (e.g. `s3://< your bucket >/models/yolov5s-640.tar.gz`)
    - make sure that the **Model name** you specify matches exactly the model name used in your Application/lambda code (it is stored in the variable/constant named **MODEL** in the current version of the Application/labmda code) *
    - select `PyTorch` as *Model framework*
    - add input with **Input name** `data` and **Input shape** `1,3,640,640`
     
- first deployment of the Application takes a bit longer due to initial conversion of the model done by AWS SageMaker Neo behind the scene. Subsequent deployments using the same model will be faster if you only change the Application code (which is usually the case)
- to troubleshoot any issues start with looking at the logs in [AWS CloudWatch](https://console.aws.amazon.com/cloudwatch). In the AWS CloudWatch Console, click on **Log Groups** under **Logs** and select a click on a link that has a name of the lambda function corresponding to your Application (something like `/aws/greengrass/Lambda/us-east-1/<your account number>/yolov5s`)

***Note:*** *code versions may change making it out of sync with comments in this notebook, always use the latest values from the code when referred to*

### What's next?
---------------------

This was just a taster to show you how to run a PyTorch based YOLOv5 model on Panorama appliance. Next logical step would be fine-tuning the pre-trained YOLOv5 model using your own dataset to recognise your own object types. Examples of doing that are available in the repository referencd earlier in this notebook.